In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.2'
# spark_version = 'spark-3.<spark version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("GT-FinalProject").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
# import and clean data frame
from pyspark import SparkFiles
url="https://aws-gt-finalproject001.s3.amazonaws.com/full_esrb_rating_data.csv"
spark.sparkContext.addFile(url)
rating_data_df = spark.read.option('header', 'true').csv(SparkFiles.get("full_esrb_rating_data.csv"), inferSchema=True, sep=',')
rating_data_df = rating_data_df.drop('_c0')
rating_data_df = rating_data_df.dropDuplicates(["title"])
rating_data_df.show(20)

In [ ]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://aws-gt-finalproject.c9qveamcfmcm.us-east-2.rds.amazonaws.com:5432/esrb_rating_data"
config = {"user":"postgres",
          "password": "PASSWORDHERE",
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to table

rating_data_df.write.jdbc(url=jdbc_url, table='esrb_rating', mode=mode, properties=config)